In [1]:
import os

from pathlib import Path

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

In [2]:
ROOT_DIR = '../'
MODEL_PATH = ROOT_DIR + 'models/Patryk-ResNeXt-optimization-model.pkt'
TRAINING_LABELS_PATH = ROOT_DIR + 'data/training_labels.csv'
TEST_DATA_PATH = ROOT_DIR + 'data/live_test_images'
SUBMISSION_FILE = ROOT_DIR + 'results/submission.csv'
OPTIMAL_THRESHOLD_PATH = ROOT_DIR + 'models/optimal_thresholds.npy'

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cpu


/home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
class TestDataset(Dataset):

    def __init__(self, img_dir, transform):
        self.transform = transform
        self.images = [os.path.join(img_dir, fname) for fname in os.listdir(img_dir)]

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)
        return image
  
    def __len__(self):
        return len(self.images)

In [4]:
model = torch.load(MODEL_PATH, map_location=DEVICE)

In [5]:
batch_size = 16
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_set = TestDataset(TEST_DATA_PATH, transform)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size)

In [6]:
results = []

for batch in test_loader:
    res = model(batch.to(DEVICE))
    res = torch.sigmoid(res).to(torch.float32).cpu().detach().numpy()
    results.append(res)

In [7]:
with open(OPTIMAL_THRESHOLD_PATH, 'rb') as f:
    optimal_thresholds = np.load(f)

In [8]:
optimal_thresholds

array([0.125     , 0.33333333, 0.17073171, 0.40366972, 0.17647059,
       0.36363636, 0.4       , 0.30952381, 0.34065934, 0.17460317,
       0.11111111, 0.26857143, 0.32038835, 0.30882353, 0.34558824,
       0.26666667, 0.3       , 0.3030303 , 0.22222222, 0.22857143,
       0.20588235, 0.20689655, 0.2       , 0.08      , 0.125     ,
       0.3880597 , 0.29275362, 0.35      , 0.20689655, 0.29299363,
       0.29090909, 0.46153846, 0.27272727, 0.2       , 0.0952381 ,
       0.42553191, 0.5       , 0.32824427])

In [9]:
df = pd.read_csv(TRAINING_LABELS_PATH)
df_test = pd.DataFrame(np.vstack(results), columns = df.columns[1:])
df_test = (df_test > optimal_thresholds).astype(int) # Threshold optimization
names = pd.DataFrame({'Name': [Path(el).parts[-1] for el in test_loader.dataset.images]})

submission = pd.concat([names, df_test], axis = 1)
submission.to_csv(SUBMISSION_FILE, index = False)